In [1]:
import os
from torch_geometric.data import Data
import torch

In [2]:
dataset = []

for ptfile in os.listdir('concept_drift_101'):
    dataset.append(torch.load(f'concept_drift_101/{ptfile}'))

In [3]:
sum=0
for i in dataset:
    sum += len(i.y)

In [4]:
sum

10710

In [5]:
train_data = dataset[0:int(0.8*366)]
test_data = dataset[int(0.8*366):]

In [6]:
dataset[1].edge_to_edge_adj_matr.count_nonzero()

tensor(40)

In [7]:
dataset[1].y

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [8]:
dataset[1].edge_attr

tensor([[0.0508, 0.8097, 0.8637, 0.5671, 0.1195, 0.0096, 0.3485, 0.6798, 0.2840,
         0.0110, 0.3701, 0.4659, 0.0744, 0.0124, 0.7723, 0.4548, 0.5860, 0.4063,
         0.0057, 0.7909, 0.5593, 0.0070, 0.7916, 0.0703, 0.7631, 0.8725, 0.3105,
         0.6588, 0.0430, 0.0138, 0.8891, 0.8529, 0.0897, 0.7732, 0.3808],
        [0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767,
         0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767,
         0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767,
         0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 

In [9]:
len(train_data)

292

In [10]:
len(test_data)

74

In [11]:
from model_nenn import Nenn

In [12]:
dataset[0].x.shape

torch.Size([6, 50])

In [13]:
dataset[0].edge_attr.shape

torch.Size([8, 27])

In [14]:
model = Nenn(6,8,6,8,2)

In [15]:
model = model.to('cpu')

In [16]:
loss = torch.nn.CrossEntropyLoss()
# TODO Undersampling Smote
# TODO Oversampling
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [17]:
dataset[0]

Data(x=[6, 50], edge_index=[2, 27], edge_attr=[8, 27], y=[27], node_to_node_adj_matr=[50, 50], node_to_edge_adj_matr=[50, 27], edge_to_node_adj_matr=[27, 50], edge_to_edge_adj_matr=[27, 27])

In [18]:
def train():
    model.train()
    # Enumerar sobre os dados.
    for epoch in range(100):
        for ts, data in enumerate(train_data):
        
            # Resetar Gradientes
            optimizer.zero_grad()
            # Passando as informações do batch e de conexão dos grafos
            logits, _ = model(
                data.x.T.type(torch.FloatTensor),
                data.edge_attr.T.type(torch.FloatTensor),
                data.edge_to_edge_adj_matr.type(torch.FloatTensor).T,
                data.edge_to_node_adj_matr.type(torch.FloatTensor).T,
                data.node_to_edge_adj_matr.type(torch.FloatTensor).T,
                data.node_to_node_adj_matr.type(torch.FloatTensor).T
            )
            #label_pred = pred.max(1)[1]
            # Calculando a perda e os gradientes.
            l = loss(logits, data.y)
            l.backward()
            # Atualizar usando os gradientes.
            optimizer.step()
            print('ts',ts+1,'epoch =', epoch + 1, 'loss =', l.item())

In [19]:
train()

ts 1 epoch = 1 loss = 0.6889553070068359
ts 2 epoch = 1 loss = 0.6753334403038025
ts 3 epoch = 1 loss = 0.6798202395439148
ts 4 epoch = 1 loss = 0.6784133315086365
ts 5 epoch = 1 loss = 0.6864336729049683
ts 6 epoch = 1 loss = 0.677548885345459
ts 7 epoch = 1 loss = 0.6842171549797058
ts 8 epoch = 1 loss = 0.6896958351135254
ts 9 epoch = 1 loss = 0.6736385226249695
ts 10 epoch = 1 loss = 0.6686111092567444
ts 11 epoch = 1 loss = 0.6878237128257751
ts 12 epoch = 1 loss = 0.6816275119781494
ts 13 epoch = 1 loss = 0.6868739128112793
ts 14 epoch = 1 loss = 0.6820669770240784
ts 15 epoch = 1 loss = 0.6730403304100037
ts 16 epoch = 1 loss = 0.6709883213043213
ts 17 epoch = 1 loss = 0.6707191467285156
ts 18 epoch = 1 loss = 0.6896634697914124
ts 19 epoch = 1 loss = 0.6716911792755127
ts 20 epoch = 1 loss = 0.6765976548194885
ts 21 epoch = 1 loss = 0.6785860657691956
ts 22 epoch = 1 loss = 0.6881234049797058
ts 23 epoch = 1 loss = 0.6728113293647766
ts 24 epoch = 1 loss = 0.6774756908416748
ts

ts 197 epoch = 1 loss = 0.6663239002227783
ts 198 epoch = 1 loss = 0.6507859826087952
ts 199 epoch = 1 loss = 0.6593735814094543
ts 200 epoch = 1 loss = 0.6157152056694031
ts 201 epoch = 1 loss = 0.6464712023735046
ts 202 epoch = 1 loss = 0.6534605622291565
ts 203 epoch = 1 loss = 0.6533543467521667
ts 204 epoch = 1 loss = 0.6389960646629333
ts 205 epoch = 1 loss = 0.6554347276687622
ts 206 epoch = 1 loss = 0.65387362241745
ts 207 epoch = 1 loss = 0.634760856628418
ts 208 epoch = 1 loss = 0.6388254761695862
ts 209 epoch = 1 loss = 0.6518175005912781
ts 210 epoch = 1 loss = 0.6413396000862122
ts 211 epoch = 1 loss = 0.662619948387146
ts 212 epoch = 1 loss = 0.6541547179222107
ts 213 epoch = 1 loss = 0.640895426273346
ts 214 epoch = 1 loss = 0.6441798210144043
ts 215 epoch = 1 loss = 0.6593895554542542
ts 216 epoch = 1 loss = 0.628849983215332
ts 217 epoch = 1 loss = 0.6183236241340637
ts 218 epoch = 1 loss = 0.6446578502655029
ts 219 epoch = 1 loss = 0.6512591242790222
ts 220 epoch = 1 

ts 104 epoch = 2 loss = 0.6134896874427795
ts 105 epoch = 2 loss = 0.5832655429840088
ts 106 epoch = 2 loss = 0.6064045429229736
ts 107 epoch = 2 loss = 0.6134443879127502
ts 108 epoch = 2 loss = 0.6165826916694641
ts 109 epoch = 2 loss = 0.6047928929328918
ts 110 epoch = 2 loss = 0.6197147369384766
ts 111 epoch = 2 loss = 0.622501015663147
ts 112 epoch = 2 loss = 0.5817692875862122
ts 113 epoch = 2 loss = 0.6212211847305298
ts 114 epoch = 2 loss = 0.6397361159324646
ts 115 epoch = 2 loss = 0.6229766011238098
ts 116 epoch = 2 loss = 0.6514539122581482
ts 117 epoch = 2 loss = 0.6294030547142029
ts 118 epoch = 2 loss = 0.6417744159698486
ts 119 epoch = 2 loss = 0.6124926209449768
ts 120 epoch = 2 loss = 0.6013215184211731
ts 121 epoch = 2 loss = 0.6205582618713379
ts 122 epoch = 2 loss = 0.6486445069313049
ts 123 epoch = 2 loss = 0.6052477359771729
ts 124 epoch = 2 loss = 0.6412005424499512
ts 125 epoch = 2 loss = 0.6282745003700256
ts 126 epoch = 2 loss = 0.5927524566650391
ts 127 epoch

KeyboardInterrupt: 

In [ ]:
label_pred_list = []
y_true_list = []

def test():
    model.eval()
    with torch.no_grad():
        global label_pred_list
        global y_true_list
        for data in test_data:
            data.to('cpu')
            logits, _ = model(
                data.x.T.type(torch.FloatTensor),
                data.edge_attr.T.type(torch.FloatTensor),
                data.edge_to_edge_adj_matr.T.type(torch.FloatTensor),
                data.edge_to_node_adj_matr.T.type(torch.FloatTensor),
                data.node_to_edge_adj_matr.T.type(torch.FloatTensor),
                data.node_to_node_adj_matr.T.type(torch.FloatTensor)
            )
            label_pred = logits.max(1)[1].tolist()
            label_pred_list += label_pred
            y_true_list += data.y.tolist()
    model.train()

In [ ]:
test()

In [ ]:
import numpy as np
np.sum(label_pred_list)

In [ ]:
label_pred_list

In [ ]:
y_true_list

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
mean_precision = []
mean_recall = []
mean_f1 = []

prec = precision_score(y_true_list, label_pred_list, average='binary', labels=[0])
rec = recall_score(y_true_list, label_pred_list, average='binary', labels=[0])
f1 = f1_score(y_true_list, label_pred_list, average='binary', labels=[0])

In [ ]:
print(classification_report(y_true_list,label_pred_list))

In [ ]:
prec

In [ ]:
rec

In [ ]:
f1

In [ ]:
prec = precision_score(y_true_list, label_pred_list, average='macro')
rec = recall_score(y_true_list, label_pred_list, average='macro')
f1 = f1_score(y_true_list, label_pred_list, average='macro')

In [ ]:
prec

In [ ]:
rec

In [ ]:
f1

# Adicionando XGBoost

In [ ]:
# Pesos saída

embeddings_train = []
y_train = []

model.eval()
with torch.no_grad():

    for data in train_data:
        data.to('cpu')
        _, embedding_train = model(
            data.x.T.type(torch.FloatTensor),
            data.edge_attr.T.type(torch.FloatTensor),
            data.edge_to_edge_adj_matr.type(torch.FloatTensor),
            data.edge_to_node_adj_matr.type(torch.FloatTensor),
            data.node_to_edge_adj_matr.type(torch.FloatTensor),
            data.node_to_node_adj_matr.type(torch.FloatTensor)
        )
        embeddings_train += embedding_train.numpy().tolist()
        y_train += data.y.numpy().tolist()
model.train()

In [ ]:
embeddings_train = np.array(embeddings_train)

In [ ]:
y_train = np.array(y_train)

In [ ]:
embeddings_train.shape

In [ ]:
y_train

In [ ]:
import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

In [ ]:
xgb_model.fit(embeddings_train, y_train)

In [ ]:
embeddings_test = []
y_true = []

model.eval()
with torch.no_grad():

    for data in test_data:
        data.to('cpu')
        _, embedding_test = model(
            data.x.T.type(torch.FloatTensor),
            data.edge_attr.T.type(torch.FloatTensor),
            data.edge_to_edge_adj_matr.type(torch.FloatTensor),
            data.edge_to_node_adj_matr.type(torch.FloatTensor),
            data.node_to_edge_adj_matr.type(torch.FloatTensor),
            data.node_to_node_adj_matr.type(torch.FloatTensor)
        )
        embeddings_test += embedding_test.numpy().tolist()
        y_true += data.y.numpy().tolist()
model.train()

In [ ]:
embeddings_test = np.array(embeddings_test)

In [ ]:
y_true = np.array(y_true)

In [ ]:
embeddings_test.shape

In [ ]:
y_true.shape

In [ ]:
y_pred = xgb_model.predict(embeddings_test)

In [ ]:
print(classification_report(y_true,y_pred))